In [ ]:
# 📦 Step 1: Install dependencies
!pip install gradio pandas requests --quiet

# 📁 Step 2: Import libraries
import gradio as gr
import requests
import pandas as pd

# 📚 Step 3: Book recommendation logic
def recommend_books(title_like, genre_filter, language_filter, year_filter):
    query = f"{title_like}"
    if genre_filter and genre_filter != "None":
        query += f"+subject:{genre_filter}"
    if language_filter:
        query += f"&langRestrict={language_filter}"

    url = f"https://www.googleapis.com/books/v1/volumes?q={query}&maxResults=10"
    response = requests.get(url)
    books = response.json().get("items", [])

    if not books:
        return "❌ No books found. Try changing filters or book title."

    results = []
    for book in books:
        info = book.get("volumeInfo", {})
        title = info.get("title", "Unknown Title")
        authors = ", ".join(info.get("authors", ["Unknown Author"]))
        published = info.get("publishedDate", "Unknown")
        description = info.get("description", "No description available.")[:200] + "..."
        thumbnail = info.get("imageLinks", {}).get("thumbnail", "")

        # Year filter check
        if year_filter and not published.startswith(year_filter):
            continue

        results.append({
            "Title": title,
            "Authors": authors,
            "Published": published,
            "Description": description,
            "Image": thumbnail
        })

    if not results:
        return "❌ No books matched the filters."

    # Convert to Markdown + images
    output = ""
    for book in results:
        output += f"![Book Cover]({book['Image']})\n" if book['Image'] else ""
        output += f"📖 **{book['Title']}**\n👤 *{book['Authors']}*\n📅 {book['Published']}\n📝 {book['Description']}\n\n"

    return output

# 🎨 Step 4: Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("# 📚 Book Recommendation System")
    gr.Markdown("Get book suggestions based on your favorites with filters!")

    with gr.Row():
        title_input = gr.Textbox(label="Book you liked (e.g., Harry Potter)")
        genre_input = gr.Dropdown(
            choices=["None", "Fantasy", "Thriller", "Romance", "Science", "Fiction", "Non-Fiction", "History", "Mystery"],
            label="Genre (Optional)",
            value="None"
        )

    with gr.Row():
        lang_input = gr.Dropdown(choices=["en", "hi", "fr", "es", "de"], label="Language (Optional)", value=None)
        year_input = gr.Textbox(label="Filter Year (e.g., 2007)", placeholder="Optional")

    btn = gr.Button("🔍 Recommend")

    output_text = gr.Markdown()

    btn.click(
        recommend_books,
        inputs=[title_input, genre_input, lang_input, year_input],
        outputs=output_text
    )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ec17d9d83103d18422.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
